In [1]:

import json
# import requests
from rdkit import Chem
import pandas as pd
from wdqs import Client



In [ ]:
df = pd.read_excel('wikidata_compound_properties.xlsx')
props = df['Property'].tolist()
accession = df['Accession'].tolist()
for i,p in enumerate(props):
    props[i] = p[:-1]


In [ ]:
client = Client()
terms = ''
for p in props:
    terms = '%s ?%s'%(terms,p)

opt_prop = ''
for i,p in enumerate(props):
    opt_prop = '%sOPTIONAL { ?compound wdt:%s ?%s } .\n'%(opt_prop,accession[i],p)

query = '''
SELECT ?compound ?compoundLabel%s
WHERE
{
    ?compound wdt:P31 wd:Q11173 .
    %s
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
}
#ORDER BY ?compoundLabel
'''%(terms,opt_prop)
result = client.query(query)


In [ ]:
df = pd.DataFrame(result)
df.to_excel('wikimedia_compound.xls')

In [ ]:
# from wdqs import Client
# client = Client()

# query = '''
# SELECT ?discoverer ?name (COUNT(?asteroid) AS ?count)
#  WHERE {
#    ?asteroid wdt:P31 wd:Q11173 .
#    ?asteroid wdt:P61 ?discoverer .

#   SERVICE wikibase:label {
#     bd:serviceParam wikibase:language "en" .
#     ?discoverer rdfs:label ?name
#    }
#  }
#  GROUP BY ?discoverer ?name 
#  ORDER BY DESC(?count)
#  LIMIT 1000
# '''
# result = client.query(query)
# result

In [ ]:
""" contribution from Hans de Winter """
def _InitialiseNeutralisationReactions():
    patts= (
        # Imidazoles
        ('[n+;H]','n'),
        # Amines
        ('[N+;!H0]','N'),
        # Carboxylic acids and alcohols
        ('[$([O-]);!$([O-][#7])]','O'),
        # Thiols
        ('[S-;X1]','S'),
        # Sulfonamides
        ('[$([N-;X2]S(=O)=O)]','N'),
        # Enamines
        ('[$([N-;X2][C,N]=C)]','N'),
        # Tetrazoles
        ('[n-]','[nH]'),
        # Sulfoxides
        ('[$([S-]=O)]','S'),
        # Amides
        ('[$([N-]C=O)]','N'),
        )
    return [(Chem.MolFromSmarts(x),Chem.MolFromSmiles(y,False)) for x,y in patts]

_reactions=None
def NeutraliseCharges(mol, reactions=None):
    global _reactions
    if reactions is None:
        if _reactions is None:
            _reactions=_InitialiseNeutralisationReactions()
        reactions=_reactions
#     mol = Chem.MolFromSmiles(smiles)
    replaced = False
    for i,(reactant, product) in enumerate(reactions):
        while mol.HasSubstructMatch(reactant):
            replaced = True
            rms = AllChem.ReplaceSubstructs(mol, reactant, product)
            rms_smiles = Chem.MolToSmiles(rms[0])
            mol = Chem.MolFromSmiles(rms_smiles)
    if replaced:
        return (mol, True) #Chem.MolToSmiles(mol,True)
    else:
        return (mol, False)

In [ ]:
# canonicalSMILES
# InChI
# PubChemCI

In [ ]:
neutral_inchi = []
metatlas_name = []
# for cc in allCompounds:
inchi = df.iloc[0]['InChI'].encode('utf-8')
smiles = df.iloc[0]['canonicalSMILES'].encode('utf-8')
if not 'InChI=' in inchi:
    inchi = '%s%s'%('InChI=',inchi)
print inchi

myMol_inchi = Chem.MolFromInchi(inchi)
myMol, neutralised = NeutraliseCharges(myMol_inchi)
print Chem.MolToInchi(myMol)

myMol_smiles = Chem.MolFromSmiles(smiles)
myMol, neutralised = NeutraliseCharges(myMol_smiles)
print Chem.MolToInchi(myMol)

#  ik.append(Chem.InchiToInchiKey(i))
#                 nb.append(myMol.GetNumBonds())
#                 charge.append(Chem.GetFormalCharge(myMol))
# # metatlas_name.append(cc.name)

# print inchi_key
# url = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/inchikey/%s/property/inchi/txt'%inchi_key
# import requests
# inchi = requests.get(url).content.strip().split('\n')[0]
# print inchi

# # http://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/857,791,94244,614,1054,247,352038,107738,588,6140,80817,9837272,444109,938,6287,33032,118276,6057,6137,673,/property/MonoisotopicMass,MolecularFormula,InChI/CSV
# import requests, json
# # url = 'http://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/857/JSON'
# url = 'http://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/857/JSON'
# r = requests.get(url)
# d = r.json()
# print d

In [44]:
from rdkit import Chem

In [2]:
# sdf_file = '/Users/bpb/Downloads/Compound_014300001_014325000.sdf'
sdf_file = '/Users/bpb/Downloads/LMSDFDownload28Jun15/LMSDFDownload28Jun15FinalAll.sdf'
# sdf_file = '/Users/bpb/Downloads/Compound_014300001_014325000.sdf.gz'
# sdf_file = '/Users/bpb/Downloads/structures.sdf'

In [76]:
suppl = Chem.SDMolSupplier(sdf_file)
mols = [m for m in suppl]
# for m in suppl:
#     print Chem.MolToInchi(m)

In [78]:
import gzip
with gzip.open(sdf_file,'rb') as inf:
    fsuppl = Chem.ForwardSDMolSupplier(inf)
    mols = [m for m in fsuppl]


In [79]:
Chem.MolToInchi(mols[0])

'InChI=1S/C8HCl3FNO/c9-6-3(8(11)14)1-5(12)7(10)4(6)2-13/h1H'

In [69]:
# df = pd.read_excel('/Users/bpb/Downloads/MetAtlas_Export_MetaCyc_Compounds.xlsx')
# df = pd.read_excel('/Users/bpb/Downloads/BML-2865.xlsx')
df = pd.read_excel('/Users/bpb/Downloads/MSMLS map - mz overlap edit sk.xlsx')
print df.shape


(638, 25)


In [3]:
from rdkit.Chem import PandasTools
import pandas as pd
# frame = PandasTools.LoadSDF(sdfFile,smilesName='SMILES',molColName='Molecule',includeFingerprints=True)
frame = PandasTools.LoadSDF(sdf_file)
print frame.shape

(40716, 23)


In [81]:
frame.keys()#to_excel('first_sdf_file.xls')

Index([u'ID', u'PUBCHEM_ATOM_DEF_STEREO_COUNT', u'PUBCHEM_ATOM_UDEF_STEREO_COUNT', u'PUBCHEM_BONDANNOTATIONS', u'PUBCHEM_BOND_DEF_STEREO_COUNT', u'PUBCHEM_BOND_UDEF_STEREO_COUNT', u'PUBCHEM_CACTVS_COMPLEXITY', u'PUBCHEM_CACTVS_HBOND_ACCEPTOR', u'PUBCHEM_CACTVS_HBOND_DONOR', u'PUBCHEM_CACTVS_ROTATABLE_BOND', u'PUBCHEM_CACTVS_SUBSKEYS', u'PUBCHEM_CACTVS_TAUTO_COUNT', u'PUBCHEM_CACTVS_TPSA', u'PUBCHEM_COMPONENT_COUNT', u'PUBCHEM_COMPOUND_CANONICALIZED', u'PUBCHEM_COMPOUND_CID', u'PUBCHEM_COORDINATE_TYPE', u'PUBCHEM_EXACT_MASS', u'PUBCHEM_HEAVY_ATOM_COUNT', u'PUBCHEM_ISOTOPIC_ATOM_COUNT', u'PUBCHEM_IUPAC_CAS_NAME', u'PUBCHEM_IUPAC_INCHI', u'PUBCHEM_IUPAC_INCHIKEY', u'PUBCHEM_IUPAC_NAME', u'PUBCHEM_IUPAC_OPENEYE_NAME', u'PUBCHEM_IUPAC_SYSTEMATIC_NAME', u'PUBCHEM_IUPAC_TRADITIONAL_NAME', u'PUBCHEM_MOLECULAR_FORMULA', u'PUBCHEM_MOLECULAR_WEIGHT', u'PUBCHEM_MONOISOTOPIC_WEIGHT', u'PUBCHEM_NONSTANDARDBOND', u'PUBCHEM_OPENEYE_CAN_SMILES', u'PUBCHEM_OPENEYE_ISO_SMILES', u'PUBCHEM_TOTAL_CHARGE', u

In [53]:
frame.drop('ROMol', axis=1, inplace=True)

In [54]:
frame.to_excel('lipidmaps_sdf_file.xls')

In [ ]:
# #Get synonyms here:
# # ftp://ftp.ncbi.nlm.nih.gov/pubchem/Compound/Extras/
#     These are listings of all names associated with a CID. The
#     unfiltered list are names aggregated from all SIDs whose 
#     standardized form is that CID, sorted by weight with the "best"
#     names first. The filtered list has some names removed that are
#     considered inconsistend with the structure. Both are gzipped text
#     files with CID, tab, and name on each line. Note that the
#     names may be composed of more than one word, separated by spaces.

In [ ]:
# df['bar'].fillna(df['foo'], inplace=True)
# del df['foo']